In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [14]:



# Load the iris dataset
df = pd.read_csv('./wine.data', header=None)
# Assign column names
df.columns = ['class', 'alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 
              'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins', 
              'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline']

# Display the first few rows of the dataframe
df.head()



,class,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [15]:
def scott_bin_width(data):
    """
    Calculate the optimal bin width using Scott's rule.
    
    Parameters:
    data (array-like): The input data for which to calculate the bin width.
    
    Returns:
    float: The optimal bin width.
    """
    n = len(data)
    sigma = np.std(data)
    bin_width = 3.5 * sigma / n**(1/3)
    return bin_width

In [16]:

# Load the wine dataset
df = pd.read_csv('./wine.data', header=None)
df.columns = ['class', 'alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 
              'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins', 
              'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline']

# Quantize each feature using Scott's rule
quantized_data = df.copy()
for column in df.columns[1:]:  # Exclude the 'class' column
    bin_width = scott_bin_width(df[column])
    num_bins = int((df[column].max() - df[column].min()) / bin_width)
    quantized_data[column] = np.floor((df[column] - df[column].min()) / bin_width).astype(int)

In [21]:
def display_caracteristics(df_encoded, df):
    # Afficher les caractéristiques du dataset
    num_features = df_encoded.shape[1] - 1
    num_instances = df_encoded.shape[0]
    num_classes = df['class'].nunique()

    print(f"Nombre de features: {num_features}")
    print(f"Nombre d'instances: {num_instances}")
    print(f"Nombre de classes: {num_classes}")



In [22]:
# Appeler la fonction avec les DataFrames appropriés
display_caracteristics(quantized_data, df)

Nombre de features: 13
Nombre d'instances: 178
Nombre de classes: 3


In [24]:
# Display the quantized data
print(quantized_data.head())

# # Optionally, plot histograms for each feature
# for column in df.columns[:-1]:
#     plt.figure()
#     plt.hist(df[column], bins=int((df[column].max() - df[column].min()) / scott_bin_width(df[column])), edgecolor='black')
#     plt.title(f"Histogram of {column} using Scott's Rule")
#     plt.xlabel(column)
#     plt.ylabel('Frequency')
#     plt.show()

   class  alcohol  malic_acid  ash  alcalinity_of_ash  magnesium  \
0      1        6           1    6                  2          6   
1      1        4           1    4                  0          3   
2      1        4           2    7                  3          3   
3      1        6           1    6                  2          4   
4      1        4           2    8                  5          5   

   total_phenols  flavanoids  nonflavanoid_phenols  proanthocyanins  \
0              4           4                     1                5   
1              4           3                     1                2   
2              4           4                     2                6   
3              7           5                     1                4   
4              4           3                     3                3   

   color_intensity  hue  od280/od315_of_diluted_wines  proline  
0                3    3                             6        4  
1                2    4           

In [25]:
# Change the classes to be indexed from 0 instead of 1
quantized_data['class'] = quantized_data['class'] - 1


In [26]:
quantized_data.head()

,class,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,0,6,1,6,2,6,4,4,1,5,3,3,6,4
1,0,4,1,4,0,3,4,3,1,2,2,4,4,3
2,0,4,2,7,3,3,4,4,2,6,3,3,4,4
3,0,6,1,6,2,4,7,5,1,4,4,2,4,6
4,0,4,2,8,5,5,4,3,3,3,2,3,3,2


In [34]:
X = quantized_data.iloc[:, 1:]
y = quantized_data.iloc[:, 0]





In [35]:
X

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,6,1,6,2,6,4,4,1,5,3,3,6,4
1,4,1,4,0,3,4,3,1,2,2,4,4,3
2,4,2,7,3,3,4,4,2,6,3,3,4,4
3,6,1,6,2,4,7,5,1,4,4,2,4,6
4,4,2,8,5,5,4,3,3,3,2,3,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,5,7,6,4,2,1,0,5,1,4,1,1,2
174,4,4,6,5,3,2,0,3,2,4,1,0,2
175,4,5,5,4,5,1,0,3,2,6,0,0,2
176,4,2,5,4,5,1,0,5,2,5,0,0,2


In [36]:
y

0      0
1      0
2      0
3      0
4      0
      ..
173    2
174    2
175    2
176    2
177    2
Name: class, Length: 178, dtype: int64

In [37]:
# Concatenate X and y
dataset = pd.concat([X, y], axis=1)

# Export the dataset to a CSV file
dataset.to_csv('./wine.csv', index=False)
